<a href="https://colab.research.google.com/github/amanda-pinheiro/sistema-de-busca-com-a-IA-Gemini/blob/main/Sistema_de_Busca_com_Gemini%2C_a_IA_Generativa_do_Google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-generativeai

In [2]:
# Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

GOOGLE_API_KEY='your_key'
genai.configure(api_key='your_key')

In [3]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [5]:
# Exemplo de Embeddings
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
              "\n"
              "Artigo completo:\n"
              "Gemini API & Google AI Studio: uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.05101426, -0.03613156, -0.019123854, 0.020116216, 0.065976396, -0.0048903166, -0.026942104, -0.018211251, 0.06330574, 0.063109465, 0.010826925, 0.013041031, -0.04629405, -0.022014992, 0.009229916, -0.02573746, 0.021653203, -0.011153807, -0.0348972, -0.0009092235, 0.009706928, 0.0067370534, -0.029662568, -0.06631569, -0.019942971, 0.021168595, 0.014706574, -0.038552426, -0.032639463, 0.026912924, -0.050172154, 0.05108427, -0.034169957, 0.009925636, -0.03678325, -0.040148664, -0.025821619, -0.04494516, -0.001540135, -0.003911065, 0.0063097286, -0.085798755, -0.009292148, 0.026569776, -0.0033266735, -0.021291452, 0.045684602, 0.043331657, 0.013354058, -0.05835676, 0.027991453, 0.024097914, 0.06539191, -0.042039778, -0.0044741207, -0.008545204, 0.0230221, -0.03276091, 0.024746697, 0.0010345585, -0.0012340171, 0.023708975, -0.016549205, 0.053203553, 0.01911424, -0.057121634, -0.04277545, -2.3555249e-05, 0.010683505, 0.049094588, 0.0094815055, 0.013434651, 0.05970488, -0.064

In [26]:
#Lista de Documentos que serão Buscados
DOCUMENT1 = {
    "Título":"Título para busca",
    "Conteúdo":"o que tem no documento"
    }

DOCUMENT2 = {
    "Título":"Título para busca",
    "Conteúdo":"o que tem no documento"
    }

documents = [DOCUMENT1, DOCUMENT2]

In [28]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Currículo QA tester (PT),"Rua Suriname, Pacaraima-RR, 69345-000"
1,CV QA Tester (EN),"Suriname St., Pacaraima – RR, Brazil"


In [16]:
model = "models/embedding-001"

In [19]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [29]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Currículo QA tester (PT),"Rua Suriname, Pacaraima-RR, 69345-000","[0.022041857, -0.054756947, -0.055347316, -0.0..."
1,CV QA Tester (EN),"Suriname St., Pacaraima – RR, Brazil","[0.004875643, -0.06666851, -0.045775298, -0.06..."


In [30]:
def gerar_e_buscar_consulta(consulta, base, movel):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [31]:
consulta = "Insira sua pergunta"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Rua Suriname, Pacaraima-RR, 69345-000


In [32]:
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5}

In [33]:
prompt = f'Reescreva esse texto de forma descontraída, sem adicionar informações que não façam parte do texto: {trecho}'

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Bora lá pra Rua Suriname, no rolê de Pacaraima! É o CEP 69345-000, pra você não se perder na balada!
